# 

In [1]:
import selenium 
import csv
import re
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options

import time

In [2]:
inxight_df = pd.read_csv("inxight_corrected_cas.csv")
filtered_df = inxight_df[inxight_df['results'] == 0]

In [3]:
filtered_df

,molecule,cas,best_match_url,results,query_url,query_type,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases
32,ACHILLEA MILLEFOLIUM,977000-16-0,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,ADAPALENE,106685-40-9,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,ALPHA-GALACTOSIDASE,7493-95-0,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,ALPHA AMYLASE,9005-82-7,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,ALPHA CAROTENE,7488-99-5,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2286,VERBENA OFFICINALIS,977000-41-1,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2300,VITAMIN F,7771-44-0,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2301,VITAMIN K,12001-79-5,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2319,XYLANASE,9025-57-4,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
pubchem_df = pd.read_csv("../PubChem Scraper/pubchem_longer_wait_final.csv")
pubchem_df.head(1)

,Original Molecule,Parsed Molecule,Link,Result Type,CAS,Deprecated CAS,Compound/Substance,Depositor Synonym List,Regular Synonym List,Synonym?
0,"1,2 PROPANEDIOL DIACETATE","1,2 PROPANEDIOL DIACETATE",https://pubchem.ncbi.nlm.nih.gov/compound/12198,RELEVANT,623-84-7,"134236-23-0, 1432741-27-9",COMPOUND,YES,YES,YES


In [5]:
# extract all second CAS values for all the molecules in ffiltered_df
pubchem_df = pubchem_df[['Parsed Molecule', 'CAS']]
pubchem_df

,Parsed Molecule,CAS
0,"1,2 PROPANEDIOL DIACETATE",623-84-7
1,1 HEXADECANOL,"36653-82-4, 36311-34-9, 124-29-8"
2,1 OCTADECANOL,"112-92-5, 26762-44-7"
3,"2,3-(2 IODOPROPYLIDENEDIOXY)PROPANOL",No CAS info
4,2 OXOGLUTARIC ACID,"328-50-7, 34410-46-3, 17091-15-5"
...,...,...
2900,ZIZYPHUS JUJUBA,No CAS info
2901,ZOLEDRONIC ACID,118072-93-8
2902,ZOLMITRIPTAN,139264-17-8
2903,ZOLPIDEM,82626-48-0


In [6]:
merged_df = filtered_df.merge(pubchem_df, how='left', left_on='molecule', right_on='Parsed Molecule')

# Drop the extra 'Parsed Molecule' column
merged_df.drop(columns=['Parsed Molecule'], inplace=True)

# Rename the 'CAS' column to 'CAS' in filtered_df
merged_df.rename(columns={'CAS_y': 'CAS'}, inplace=True)

# Drop the extra 'CAS' column from pubchem_df
# merged_df.drop(columns=['CAS_x'], inplace=True)

In [7]:
merged_df

,molecule,cas,best_match_url,results,query_url,query_type,Highest Development Event Status and Year,Highest Development Event Source ID,Highest Development Event Source URL,Earliest Approved Event Status and Year,Earliest Approved Event Source ID,Earliest Approved Event Source URL,conditions,phases,CAS
0,ACHILLEA MILLEFOLIUM,977000-16-0,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"977000-16-0, 8022-07-9"
1,ADAPALENE,106685-40-9,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106685-40-9
2,ALPHA-GALACTOSIDASE,7493-95-0,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7493-95-0
3,ALPHA AMYLASE,9005-82-7,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"9005-82-7, 6401-81-6, 1109-28-0"
4,ALPHA CAROTENE,7488-99-5,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7488-99-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,VERBENA OFFICINALIS,977000-41-1,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,977000-41-1
142,VITAMIN F,7771-44-0,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"7771-44-0, 11006-87-4, 506-32-1"
143,VITAMIN K,12001-79-5,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"12001-79-5, 27696-10-2, 81818-54-4"
144,XYLANASE,9025-57-4,NaN,0,https://drugs.ncats.io/substances?q=(root_code...,ONLY CAS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"9025-57-4, 7554-16-7"


In [8]:
merged_filtered_df = merged_df[['molecule', 'CAS']]
merged_filtered_df

,molecule,CAS
0,ACHILLEA MILLEFOLIUM,"977000-16-0, 8022-07-9"
1,ADAPALENE,106685-40-9
2,ALPHA-GALACTOSIDASE,7493-95-0
3,ALPHA AMYLASE,"9005-82-7, 6401-81-6, 1109-28-0"
4,ALPHA CAROTENE,7488-99-5
...,...,...
141,VERBENA OFFICINALIS,977000-41-1
142,VITAMIN F,"7771-44-0, 11006-87-4, 506-32-1"
143,VITAMIN K,"12001-79-5, 27696-10-2, 81818-54-4"
144,XYLANASE,"9025-57-4, 7554-16-7"


In [9]:
csv = merged_filtered_df.to_csv("correct_zero_results.csv")